In [9]:
from train_nep.config import get_nep_config
from train_nep.dataset import StructureDataset, collate_fn
from train_nep.model import NEP
from train_nep.alchemy import Alchemy
from torch.utils.data import DataLoader

config = get_nep_config()

train_dataset = StructureDataset(filepath="../Repository/PbTe/train.xyz", para=config)
train_loader = DataLoader(train_dataset, batch_size=5, shuffle=False, collate_fn=collate_fn)

model = NEP(config)

alchemy = Alchemy(
    model=model,
    training_set=train_loader,
    save_path="../Repository/PbTe/nep_model.pt"
)

alchemy.fit(epochs=100)
alchemy.load("../Repository/PbTe/nep_model.pt")

开始训练，总共 100 轮
设备: cpu
模型参数数量: 1773
--------------------------------------------------
tensor([0.1827, 0.1827, 0.1856, 0.1820, 0.1890], grad_fn=<DivBackward0>)
tensor([-3.7488, -3.7790, -3.7491, -3.7814, -3.7506])
  [Batch 1] Loss = 4.204020 | Energy = 3.946185 | Forces = 0.257835 | Virial = 0.000000
tensor([0.1383, 0.1352, 0.1315, 0.1421, 0.1366], grad_fn=<DivBackward0>)
tensor([-3.7478, -3.7705, -3.7946, -3.7462, -3.7721])
  [Batch 2] Loss = 4.151825 | Energy = 3.902976 | Forces = 0.248850 | Virial = 0.000000
tensor([0.0919, 0.1008, 0.0984, 0.0772, 0.0756], grad_fn=<DivBackward0>)
tensor([-3.7841, -3.7238, -3.7465, -3.8334, -3.8335])
  [Batch 3] Loss = 4.058554 | Energy = 3.873057 | Forces = 0.185497 | Virial = 0.000000
tensor([0.0504, 0.0429, 0.0645, 0.0351, 0.0367], grad_fn=<DivBackward0>)
tensor([-3.7733, -3.7945, -3.7167, -3.8334, -3.8334])
  [Batch 4] Loss = 4.020134 | Energy = 3.836168 | Forces = 0.183967 | Virial = 0.000000
tensor([ 0.0048,  0.0156,  0.0005, -0.0019,  0.0132], 

Loss/train_energy,██▇▇▆▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Loss/train_forces,▁▂▃▄▅██▇▆▅▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Loss/train_total,█▇▇▆▆▅▅▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Loss/train_virial,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▇▇▇▇▇▇▇▇█
Loss/train_energy,0.01162
Loss/train_forces,0.20153
Loss/train_total,0.21315
Loss/train_virial,0
epoch,100



训练完成!
最佳验证损失: inf
✓ 已加载检查点: ../Repository/PbTe/nep_model.pt


In [10]:
import torch
from wizard.io import read_xyz
from train_nep.interface import NEPCalculator
from train_nep.config import get_nep_config

test_atoms = read_xyz("../Repository/PbTe/train.xyz")[0]  

para = get_nep_config()  
nep_calc = NEPCalculator(model_path="../Repository/PbTe/nep_model.pt", para=para) 
test_atoms.calc = nep_calc

pred_energy = test_atoms.get_potential_energy()
pred_forces = test_atoms.get_forces()

print(f"N_atoms: {len(test_atoms)} atoms")
print(f"Chemical formula: {test_atoms.get_chemical_formula()}")
print(f"Predicted energy: {pred_energy:.6f} eV")
print(f"Energy per atom: {pred_energy/len(test_atoms):.6f} eV/atom")
print(f"Forces shape: {pred_forces.shape}")

# 与 DFT 比较
if 'energy' in test_atoms.info:
    dft_energy = test_atoms.info['energy']
    print(f"\nComparison with true (DFT) values:")
    print(f"Energy: {dft_energy/len(test_atoms):.6f} eV")
    print(f"Energy error: {abs(pred_energy - dft_energy)/len(test_atoms):.6f} eV/atom")

if 'forces' in test_atoms.info:
    dft_forces = test_atoms.info['forces']
    diff = torch.tensor(pred_forces - dft_forces)  # [N,3]
    mae_force = diff.abs().mean()
    print(f"Force MAE: {mae_force:.6f} eV/Å")

N_atoms: 250 atoms
Chemical formula: Pb125Te125
Predicted energy: -936.351624 eV
Energy per atom: -3.745406 eV/atom
Forces shape: (250, 3)

Comparison with true (DFT) values:
Energy: -3.748764 eV
Energy error: 0.003358 eV/atom
Force MAE: 0.262966 eV/Å
